# Навчання RL для балансування маятника на візку

Цей блокнот є частиною [навчальної програми "AI для початківців"](http://aka.ms/ai-beginners). Він був натхненний [офіційним підручником PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) та [цією реалізацією Cartpole на PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

У цьому прикладі ми використаємо RL для навчання моделі балансувати маятник на візку, який може рухатися вліво та вправо по горизонтальній шкалі. Ми будемо використовувати середовище [OpenAI Gym](https://www.gymlibrary.ml/) для симуляції маятника.

> **Note**: Ви можете запускати код цього уроку локально (наприклад, у Visual Studio Code), у такому випадку симуляція відкриється в новому вікні. Якщо ви запускаєте код онлайн, можливо, вам доведеться внести деякі зміни до коду, як описано [тут](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Ми почнемо з перевірки, чи встановлений Gym:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Тепер створимо середовище CartPole і подивимося, як з ним працювати. Середовище має такі властивості:

* **Action space** — набір можливих дій, які ми можемо виконувати на кожному кроці симуляції
* **Observation space** — простір спостережень, які ми можемо отримати


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Давайте подивимося, як працює симуляція. Наступний цикл запускає симуляцію, поки `env.step` не поверне прапорець завершення `done`. Ми будемо випадково вибирати дії за допомогою `env.action_space.sample()`, що означає, що експеримент, ймовірно, дуже швидко завершиться невдачею (середовище CartPole завершується, коли швидкість CartPole, його положення або кут виходять за певні межі).

> Симуляція відкриється в новому вікні. Ви можете запустити код кілька разів і побачити, як він поводиться.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Ви можете помітити, що спостереження містять 4 числа. Це:
- Позиція візка
- Швидкість візка
- Кут нахилу стрижня
- Швидкість обертання стрижня

`rew` — це винагорода, яку ми отримуємо на кожному кроці. Ви можете побачити, що в середовищі CartPole ви отримуєте 1 бал за кожен крок симуляції, і мета полягає в тому, щоб максимізувати загальну винагороду, тобто час, протягом якого CartPole може балансувати, не падаючи.

Під час навчання з підкріпленням наша мета — навчити **політику** $\pi$, яка для кожного стану $s$ буде вказувати, яку дію $a$ потрібно виконати, тобто, по суті, $a = \pi(s)$.

Якщо ви хочете ймовірнісне рішення, ви можете уявити політику як таку, що повертає набір ймовірностей для кожної дії, тобто $\pi(a|s)$ означатиме ймовірність того, що ми повинні виконати дію $a$ у стані $s$.

## Метод градієнта політики

У найпростішому алгоритмі навчання з підкріпленням, який називається **Градієнт політики**, ми будемо навчати нейронну мережу передбачати наступну дію.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Ми будемо тренувати мережу, проводячи багато експериментів і оновлюючи нашу мережу після кожного запуску. Давайте визначимо функцію, яка буде проводити експеримент і повертати результати (так званий **трейс**) - всі стани, дії (і їх рекомендовані ймовірності) та винагороди:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Ви можете запустити один епізод з ненавченим мережевим і спостерігати, що загальна винагорода (тобто тривалість епізоду) дуже низька:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Один із складних аспектів алгоритму градієнтної політики — використання **знижених винагород**. Ідея полягає в тому, що ми обчислюємо вектор загальних винагород на кожному кроці гри, і під час цього процесу знижуємо ранні винагороди, використовуючи деякий коефіцієнт $gamma$. Ми також нормалізуємо отриманий вектор, оскільки будемо використовувати його як вагу для впливу на наше навчання:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Тепер почнемо справжнє навчання! Ми проведемо 300 епізодів, і в кожному епізоді виконаємо наступне:

1. Запустимо експеримент і зберемо трасу.
2. Обчислимо різницю (`gradients`) між виконаними діями та передбаченими ймовірностями. Чим менша різниця, тим більше ми впевнені, що обрали правильну дію.
3. Розрахуємо дисконтовані винагороди та помножимо градієнти на дисконтовані винагороди — це забезпечить, що кроки з вищими винагородами матимуть більший вплив на кінцевий результат, ніж ті, що мають нижчі винагороди.
4. Очікувані цільові дії для нашої нейронної мережі частково будуть взяті з передбачених ймовірностей під час виконання, а частково — з розрахованих градієнтів. Ми використаємо параметр `alpha`, щоб визначити, наскільки враховуються градієнти та винагороди — це називається *швидкістю навчання* алгоритму підкріплення.
5. Нарешті, ми тренуємо нашу мережу на станах і очікуваних діях, і повторюємо процес.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Тепер давайте запустимо епізод із рендерингом, щоб побачити результат:


In [ ]:
_ = run_episode(render=True)

Сподіваємося, тепер ви бачите, що жердина може досить добре балансувати!

## Модель Actor-Critic

Модель Actor-Critic є подальшим розвитком градієнтів політики, у якій ми створюємо нейронну мережу для навчання як політики, так і оцінених винагород. Мережа матиме два виходи (або можна розглядати це як дві окремі мережі):
* **Actor** рекомендуватиме дію, яку слід виконати, надаючи нам розподіл ймовірностей стану, як у моделі градієнтів політики.
* **Critic** оцінюватиме, якою буде винагорода від цих дій. Він повертає загальну оцінену винагороду в майбутньому для даного стану.

Давайте визначимо таку модель:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Нам потрібно трохи змінити наші функції `discounted_rewards` та `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Тепер ми запустимо основний цикл навчання. Ми будемо використовувати процес ручного навчання мережі, обчислюючи відповідні функції втрат і оновлюючи параметри мережі:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Основні висновки

У цьому демонстраційному прикладі ми розглянули два алгоритми навчання з підкріпленням: простий градієнт політики та більш складний actor-critic. Ви могли помітити, що ці алгоритми працюють з абстрактними поняттями стану, дії та винагороди – тому їх можна застосовувати до дуже різних середовищ.

Навчання з підкріпленням дозволяє нам знаходити найкращу стратегію для розв'язання задачі, орієнтуючись лише на кінцеву винагороду. Те, що нам не потрібні розмічені набори даних, дає змогу багаторазово повторювати симуляції для оптимізації наших моделей. Однак у навчанні з підкріпленням все ще існує багато викликів, які ви зможете вивчити, якщо вирішите глибше зануритися в цю захопливу галузь штучного інтелекту.



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
